# TEST 1 : premier tests de BM25 et BERT

In [49]:
! pip install PyPDF2
! pip install rank_bm25
! pip install transformers
! pip install torch 
! pip install nltk
! pip install scikit-learn


ERROR: Invalid requirement: '#'


In [89]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text


In [90]:
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def bm25_rank(query, documents):
    # Tokenisation des documents
    tokenized_docs = [word_tokenize(doc.lower(), language='french') for doc in documents]
    
    # Initialisation de BM25
    bm25 = BM25Okapi(tokenized_docs)
    
    # Tokenisation de la requête
    tokenized_query = word_tokenize(query.lower(), language='french')
    
    # Calcul des scores BM25
    scores = bm25.get_scores(tokenized_query)
    
    # Création d'une liste de tuples (indice du document, score)
    doc_scores = list(enumerate(scores))
    
    # Tri des documents en fonction des scores (du plus élevé au plus faible)
    ranked_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_score = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_score



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [91]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def bert_rank(query, documents):
    # Chargement du modèle et du tokenizer pré-entraînés
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    
    # Fonction pour obtenir les embeddings
    def get_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:,0,:].detach().numpy()
        return embeddings[0]
    
    # Calcul des embeddings des documents
    doc_embeddings = np.array([get_embedding(doc) for doc in documents])
    
    # Calcul de l'embedding de la requête
    query_embedding = get_embedding(query)
    
    # Calcul de la similarité cosinus
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # Création d'une liste de tuples (indice du document, similarité)
    doc_similarities = list(enumerate(similarities))
    
    # Tri des documents en fonction des similarités (du plus élevé au plus faible)
    ranked_docs = sorted(doc_similarities, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_similarity = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_similarity



In [92]:

def main():
    pdf_paths = [
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf',
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf'
    ]
    
    documents = [extract_text_from_pdf(pdf_path) for pdf_path in pdf_paths]

    query = input("Entrez votre requête : ")

    print("\nRequête :", query)
    
    # Classement avec BM25
    bm25_ranked_docs, bm25_best_doc_index, bm25_best_doc_score = bm25_rank(query, documents)
    print("\nClassement des documents avec BM25 :")
    for idx, score in bm25_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Score : {score:.2f}")
    print(f"\nMeilleur document selon BM25 : {pdf_paths[bm25_best_doc_index]}, Score : {bm25_best_doc_score:.2f}")
    
    # Classement avec BERT
    bert_ranked_docs, bert_best_doc_index, bert_best_doc_similarity = bert_rank(query, documents)
    print("\nClassement des documents avec BERT :")
    for idx, similarity in bert_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Similarité : {similarity:.4f}")
    print(f"\nMeilleur document selon BERT : {pdf_paths[bert_best_doc_index]}, Similarité : {bert_best_doc_similarity:.4f}")



In [93]:
main()


Requête : Combien d’heures au maximum on peut travailler par semaine ?

Classement des documents avec BM25 :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -2.87
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Score : -2.90

Meilleur document selon BM25 : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -2.87

Classement des documents avec BERT :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Similarité : 0.7807
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.6591

Meilleur document selon BERT : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Similarité : 0.7807


# TEST 2 : Ajout de la fonctionnalité de synonyme pour améliorer BM25

Utilisation du modèle BERT en français :
Dans la fonction bert_rank, nous avons modifié le modèle pour utiliser un modèle BERT pré-entraîné en français, tel que CamemBERT


Les modèles BERT pré-entraînés en anglais (bert-base-uncased) ne sont pas optimaux pour traiter des textes en français.
CamemBERT est un modèle BERT adapté au français et donnera de meilleurs résultats sur des textes français.
Gestion des synonymes en français 



Nous utilisons le WordNet multilingue via NLTK pour obtenir des synonymes en français.
La variable lang='fra' est utilisée pour spécifier que nous souhaitons des synonymes en français.



En ajoutant des synonymes aux documents, nous enrichissons leur contenu sémantique.
Cela permet à BM25, qui est basé sur la correspondance exacte de termes, de mieux capturer les relations sémantiques entre la requête et les documents.

Certains mots peuvent ne pas avoir de synonymes dans WordNet.
Les synonymes peuvent inclure des mots qui sont des stop words ; nous avons filtré les tokens non alphabétiques mais pas les stop words parmi les synonymes pour conserver la diversité.

In [63]:
import PyPDF2
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [99]:

def add_synonyms_to_documents(documents, n):
    # Obtenir les stop words français
    stop_words = set(stopwords.words('french'))
    
    # Traiter chaque document
    new_documents = []
    for doc in documents:
        # Tokeniser le document
        tokens = word_tokenize(doc, language='french')
        
        # Identifier les mots complexes (non stop words)
        complex_words = [word for word in tokens if word.lower() not in stop_words and word.isalpha()]
        
        # Pour chaque mot complexe, trouver des synonymes
        synonyms = []
        for word in set(complex_words):  # Utiliser set pour éviter les doublons
            synsets = wn.synsets(word, lang='fra')
            # Collecter les synonymes des synsets
            syns = set()
            for synset in synsets:
                for lemma in synset.lemma_names('fra'):
                    if lemma != word:
                        syns.add(lemma.replace('_', ' '))
            # Ajouter jusqu'à 3 synonymes
            syns = list(syns)[:n]
            synonyms.extend(syns)
        
        # Ajouter les synonymes à la fin du document
        extended_doc = doc + ' ' + ' '.join(synonyms)
        new_documents.append(extended_doc)
    return new_documents

In [ ]:
def bm25_rank(query, documents):
    # Tokenisation des documents
    tokenized_docs = [word_tokenize(doc.lower(), language='french') for doc in documents]
    
    # Initialisation de BM25
    bm25 = BM25Okapi(tokenized_docs, k1=1.5, b=0.5)
    
    # Tokenisation de la requête
    tokenized_query = word_tokenize(query.lower(), language='french')
    
    # Calcul des scores BM25
    scores = bm25.get_scores(tokenized_query)
    
    # Création d'une liste de tuples (indice du document, score)
    doc_scores = list(enumerate(scores))
    
    # Tri des documents en fonction des scores (du plus élevé au plus faible)
    ranked_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_score = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_score

In [101]:
def bert_rank(query, documents):
    # Chargement du modèle et du tokenizer pré-entraînés
    tokenizer = AutoTokenizer.from_pretrained('camembert-base')
    model = AutoModel.from_pretrained('camembert-base')
    
    # Fonction pour obtenir les embeddings
    def get_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:,0,:].detach().numpy()
        return embeddings[0]
    
    # Calcul des embeddings des documents
    doc_embeddings = np.array([get_embedding(doc) for doc in documents])
    
    # Calcul de l'embedding de la requête
    query_embedding = get_embedding(query)
    
    # Calcul de la similarité cosinus
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # Création d'une liste de tuples (indice du document, similarité)
    doc_similarities = list(enumerate(similarities))
    
    # Tri des documents en fonction des similarités (du plus élevé au plus faible)
    ranked_docs = sorted(doc_similarities, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_similarity = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_similarity

In [110]:

def main_avec_synonyms(n):
    # Chemins vers vos documents PDF
    pdf_paths = [
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf',
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf'
    ]
    
    # Extraction du texte des PDFs
    documents = [extract_text_from_pdf(pdf_path) for pdf_path in pdf_paths]
    
    # Ajouter les synonymes aux documents pour BM25
    documents_with_synonyms = add_synonyms_to_documents(documents,n)
    
    # Votre requête
    query = input("Entrez votre requête : ")

    print("\nRequête :", query)
    
    # Classement avec BM25 (documents avec synonymes)
    bm25_ranked_docs, bm25_best_doc_index, bm25_best_doc_score = bm25_rank(query, documents_with_synonyms)
    print("\nClassement des documents avec BM25 :")
    for idx, score in bm25_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Score : {score:.2f}")
    print(f"\nMeilleur document selon BM25 : {pdf_paths[bm25_best_doc_index]}, Score : {bm25_best_doc_score:.2f}")
    
    # Classement avec BERT (documents originaux)
    bert_ranked_docs, bert_best_doc_index, bert_best_doc_similarity = bert_rank(query, documents)
    print("\nClassement des documents avec BERT :")
    for idx, similarity in bert_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Similarité : {similarity:.4f}")
    print(f"\nMeilleur document selon BERT : {pdf_paths[bert_best_doc_index]}, Similarité : {bert_best_doc_similarity:.4f}")




avec 3 synonymes

In [111]:
main_avec_synonyms(3)


Requête : Combien d’heures au maximum on peut travailler par semaine ?

Classement des documents avec BM25 :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -3.76
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Score : -3.77

Meilleur document selon BM25 : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -3.76

Classement des documents avec BERT :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.8213
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Similarité : 0.8124

Meilleur document selon BERT : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.82

avec 6 synonyme

In [105]:
main_avec_synonyms(6)


Requête : Combien d’heures au maximum on peut travailler par semaine ?

Classement des documents avec BM25 :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -4.22
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Score : -4.23

Meilleur document selon BM25 : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -4.22

Classement des documents avec BERT :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.8213
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Similarité : 0.8124

Meilleur document selon BERT : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.82

# TEST 3 : test avec ajout des synonyme à la query et ajout des synonyme après chaque mots

In [ ]:
import PyPDF2
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Définir les stop words français
stop_words = set(stopwords.words('french'))

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def get_synonyms(word):
    synsets = wn.synsets(word, lang='fra')
    syns = set()
    for synset in synsets:
        for lemma in synset.lemma_names('fra'):
            if lemma != word:
                syns.add(lemma.replace('_', ' '))
    return list(syns)[:3]

def add_synonyms_inline(documents):
    # Obtenir les stop words français
    stop_words = set(stopwords.words('french'))
    new_documents = []
    for doc in documents:
        tokens = word_tokenize(doc, language='french')
        new_tokens = []
        for word in tokens:
            new_tokens.append(word)
            if word.lower() not in stop_words and word.isalpha():
                synonyms = get_synonyms(word)
                new_tokens.extend(synonyms)
        new_doc = ' '.join(new_tokens)
        new_documents.append(new_doc)
    return new_documents


def expand_query(query):
    tokens = word_tokenize(query, language='french')
    expanded_tokens = []
    for word in tokens:
        expanded_tokens.append(word)
        if word.lower() not in stop_words and word.isalpha():
            synonyms = get_synonyms(word)
            expanded_tokens.extend(synonyms)
    return ' '.join(expanded_tokens)

def bm25_rank(query, documents):
    # Tokenisation des documents
    tokenized_docs = [word_tokenize(doc.lower(), language='french') for doc in documents]
    
    # Initialisation de BM25 avec des paramètres ajustés (optionnel)
    bm25 = BM25Okapi(tokenized_docs)
    
    # Tokenisation de la requête
    tokenized_query = word_tokenize(query.lower(), language='french')
    
    # Calcul des scores BM25
    scores = bm25.get_scores(tokenized_query)
    
    # Création d'une liste de tuples (indice du document, score)
    doc_scores = list(enumerate(scores))
    
    # Tri des documents en fonction des scores (du plus élevé au plus faible)
    ranked_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_score = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_score

def bert_rank(query, documents):
    # Chargement du modèle et du tokenizer pré-entraînés pour le français
    tokenizer = AutoTokenizer.from_pretrained('camembert-base')
    model = AutoModel.from_pretrained('camembert-base')
    
    # Fonction pour obtenir les embeddings
    def get_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:,0,:].detach().numpy()
        return embeddings[0]
    
    # Calcul des embeddings des documents
    doc_embeddings = np.array([get_embedding(doc) for doc in documents])
    
    # Calcul de l'embedding de la requête
    query_embedding = get_embedding(query)
    
    # Calcul de la similarité cosinus
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # Création d'une liste de tuples (indice du document, similarité)
    doc_similarities = list(enumerate(similarities))
    
    # Tri des documents en fonction des similarités (du plus élevé au plus faible)
    ranked_docs = sorted(doc_similarities, key=lambda x: x[1], reverse=True)
    
    # Récupération du meilleur document
    best_doc_index = ranked_docs[0][0]
    best_doc_similarity = ranked_docs[0][1]
    
    return ranked_docs, best_doc_index, best_doc_similarity

def main():
    # Chemins vers vos documents PDF
    pdf_paths = [
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf',
        r'C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf'
    ]
    
    # Extraction du texte des PDFs
    documents = [extract_text_from_pdf(pdf_path) for pdf_path in pdf_paths]

    documents_synonyms = add_synonyms_inline(documents)
    
    # Votre requête
    query = input("Entrez votre requête : ")
    
    # Étendre la requête avec des synonymes
    expanded_query = expand_query(query)
    
    # Classement avec BM25 (documents originaux, requête étendue)
    bm25_ranked_docs, bm25_best_doc_index, bm25_best_doc_score = bm25_rank(expanded_query, documents)
    print("Classement des documents avec BM25 :")
    for idx, score in bm25_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Score : {score:.2f}")
    print(f"\nMeilleur document selon BM25 : {pdf_paths[bm25_best_doc_index]}, Score : {bm25_best_doc_score:.2f}")
    
    # Classement avec BERT (documents originaux)
    bert_ranked_docs, bert_best_doc_index, bert_best_doc_similarity = bert_rank(query, documents)
    print("\nClassement des documents avec BERT :")
    for idx, similarity in bert_ranked_docs:
        print(f"Document : {pdf_paths[idx]}, Similarité : {similarity:.4f}")
    print(f"\nMeilleur document selon BERT : {pdf_paths[bert_best_doc_index]}, Similarité : {bert_best_doc_similarity:.4f}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Classement des documents avec BM25 :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -3.49
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Score : -3.51

Meilleur document selon BM25 : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Score : -3.49

Classement des documents avec BERT :
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.8213
Document : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION PAIE MAJ 09-24.pdf, Similarité : 0.8124

Meilleur document selon BERT : C:\Users\guill\OneDrive\Documents\ESILV\A4\Parcours recherche\Sujet\Doc_pdf\FORMATION SOCIALE MAJ 09-24.pdf, Similarité : 0.8213


Conclusion

Test avec ajout des synonyme à la fin du doc : moins bon

test avec synonyme après chaque mots : moins bon

test avec synonyme dans la requete : moins bon

test avec synonyme dans requete + après chaque mots : vraiment moins bon